In [1]:
from statsforecast.core import StatsForecast
from statsforecast.models import ( 
    AutoARIMA,
    AutoETS,
)
from statsforecast.utils import generate_series

n_series = 4
horizon = 7

series = generate_series(n_series)

sf = StatsForecast(
    models=[AutoETS(season_length=7)],
    freq='D',
)

sf.cross_validation(df=series, h=horizon, step_size = 24,
    n_windows = 2, level=[90]).head()

/Users/user/Documents/miniconda3/envs/statsforecast/lib/python3.10/site-packages/statsforecast/core.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


,ds,cutoff,y,AutoETS,AutoETS-lo-90,AutoETS-hi-90
unique_id,,,,,,
0,2000-07-10,2000-07-09,2.472186,2.264802,2.029021,2.500583
0,2000-07-11,2000-07-09,3.369775,3.207784,2.972003,3.443565
0,2000-07-12,2000-07-09,4.245229,4.248131,4.012350,4.483912
0,2000-07-13,2000-07-09,5.113708,5.267366,5.031586,5.503148
0,2000-07-14,2000-07-09,6.127178,6.203137,5.967356,6.438918


In [8]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# Make unique_id a column
series = series.reset_index()
series['unique_id'] = series['unique_id'].astype(str)

# Convert to Spark
sdf = spark.createDataFrame(series)

# Returns a Spark DataFrame
sf.cross_validation(df=sdf, h=horizon, step_size = 24,
    n_windows = 2, level=[90]).show()

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

/Users/user/Documents/miniconda3/envs/statsforecast/lib/python3.10/site-packages/pyspark/bin/spark-class: line 97: CMD: bad array subscript
head: illegal line count -- -1


RuntimeError: Java gateway process exited before sending its port number

In [4]:
import dask.dataframe as dd
from dask.distributed import Client
from fugue_dask import DaskExecutionEngine
from statsforecast import StatsForecast
from statsforecast.models import Naive
from statsforecast.utils import generate_series

# Generate Synthetic Panel Data
df = generate_series(10).reset_index()
df['unique_id'] = df['unique_id'].astype(str)
df = dd.from_pandas(df, npartitions=10)

# Instantiate FugueBackend with DaskExecutionEngine
dask_client = Client()
engine = DaskExecutionEngine(dask_client=dask_client)

/Users/user/Documents/miniconda3/envs/statsforecast/lib/python3.10/site-packages/distributed/client.py:1401: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| numpy   | 1.24.3 | 1.24.3    | 1.24.4  |
| pandas  | 1.5.3  | 1.5.3     | 2.0.3   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


In [10]:
#sf = StatsForecast(models=[Naive()], freq='D')

sf = StatsForecast(
    models=[AutoETS(season_length=7)],
    freq='D',
)

In [11]:
# Distributed predictions with FugueBackend.
sf.forecast(df=df, h=12).compute()

make fugue


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


,unique_id,ds,AutoETS
0,3,2000-08-30,0.225319
1,3,2000-08-31,1.208065
2,3,2000-09-01,2.269487
3,3,2000-09-02,3.238179
4,3,2000-09-03,4.288004
...,...,...,...
7,8,2000-03-07,1.303355
8,8,2000-03-08,2.365429
9,8,2000-03-09,3.187764
10,8,2000-03-10,4.180392


In [12]:
# Distributed cross-validation with FugueBackend.
sf.cross_validation(df=df, h=12, n_windows=2).compute()

make fugue


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


,unique_id,ds,cutoff,y,AutoETS
0,3,2000-08-17,2000-08-16,1.375511,1.186747
1,3,2000-08-18,2000-08-16,2.463606,2.245214
2,3,2000-08-19,2000-08-16,3.014476,3.226089
3,3,2000-08-20,2000-08-16,4.447845,4.256207
4,3,2000-08-21,2000-08-16,5.196284,5.236430
...,...,...,...,...,...
19,8,2000-02-24,2000-02-16,3.267941,3.172037
20,8,2000-02-25,2000-02-16,4.333817,4.156282
21,8,2000-02-26,2000-02-16,5.410231,5.274494
22,8,2000-02-27,2000-02-16,6.115387,6.290318


In [14]:
sf.cross_validation(df=df, h=horizon, step_size = 24,
    n_windows = 2, level=[90]).compute()

make fugue


<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast
<__array_function__ internals>:200: RuntimeWarning: invalid value encountered in cast


,unique_id,ds,cutoff,y,AutoETS
0,3,2000-07-30,2000-07-29,4.040266,4.262749
1,3,2000-07-31,2000-07-29,5.042655,5.252001
2,3,2000-08-01,2000-07-29,6.110698,6.221734
3,3,2000-08-02,2000-07-29,0.050007,0.219581
4,3,2000-08-03,2000-07-29,1.132520,1.187785
...,...,...,...,...,...
9,8,2000-02-24,2000-02-21,3.267941,3.175563
10,8,2000-02-25,2000-02-21,4.333817,4.159091
11,8,2000-02-26,2000-02-21,5.410231,5.242876
12,8,2000-02-27,2000-02-21,6.115387,6.266238


In [16]:
import ray
import logging
ray.init(logging_level=logging.ERROR)

series = series.reset_index()
series['unique_id'] = series['unique_id'].astype(str)
ctx = ray.data.context.DatasetContext.get_current()
ctx.use_streaming_executor = False
ray_series = ray.data.from_pandas(series).repartition(2)

2023-08-26 14:58:09,825	ERROR services.py:1197 -- Failed to start the dashboard , return code 1
2023-08-26 14:58:09,827	ERROR services.py:1222 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/ray-logging.html#logging-directory-structure' to find where the log file is.
2023-08-26 14:58:09,827	ERROR services.py:1232 -- Couldn't read dashboard.log file. Error: [Errno 2] No such file or directory: '/tmp/ray/session_2023-08-26_14-58-08_654292_78993/logs/dashboard.log'. It means the dashboard is broken even before it initializes the logger (mostly dependency issues). Reading the dashboard.err file which contains stdout/stderr.
2023-08-26 14:58:09,833	ERROR services.py:1266 -- 
The last 20 lines of /tmp/ray/session_2023-08-26_14-58-08_654292_78993/logs/dashboard.err (it contains the error message from the dashboard): 
  File "/Users/user/Documents/miniconda3/envs/statsforecast/lib/

RuntimeError: b'GCS has started but no raylets have registered yet.'